In [ ]:
%matplotlib inline
import numpy as np
import plotly.io as pio
from misc import *
from interplot import *
import plotly.express as px


## Import/generate Data

In [ ]:
"""load saved trajectories data for npz file
"""
SEQ = "PT0"
# SEQ = "PT3"

# multiple trajectories
fnpz_data = "data/helix_assoc/helix_assoc_{}_multrj_100epoch_py.npz".format(SEQ)
data_npz = np.load(fnpz_data)

# asssign data to variables
for var in data_npz.files:
     locals()[var] = data_npz[var]

# recover full data based on coord_id, indices, and unique data
SIMS_adj = SIMS_adj_uniq[coord_id_S]
SIMS_scar = SIMS_scar_uniq[coord_id_S]
SIMS_G = SIMS_G_uniq[coord_id_S]
SIMS_pair = SIMS_pair_uniq[coord_id_S]

print(SIMS_T.shape,SIMS_HT.shape,SIMS_HT_uniq.shape)
print(SIMS_adj.shape,SIMS_scar.shape,SIMS_G.shape,SIMS_HT.shape,SIMS_pair.shape)
print(SIMS_adj_uniq.shape,SIMS_scar_uniq.shape,SIMS_G_uniq.shape,SIMS_pair_uniq.shape) 
print(SIMS_dict.shape,SIMS_dict_uniq.shape)
print(coord_id_S.shape,indices_S.shape,trj_id.shape,data_embed.shape,occ_density_S.shape)
print(pca_coords.shape,pca_all_coords.shape)
print(phate_coords.shape,phate_all_coords.shape)
print(umap_coord_2d.shape,umap_all_coord_2d.shape,umap_coord_3d.shape,umap_all_coord_3d.shape)
print(tsne_coord_2d.shape,tsne_all_coord_2d.shape,tsne_coord_3d.shape,tsne_all_coord_3d.shape)

In [ ]:
"""make pd dataframe for plotting
"""
df = pd.DataFrame(data={
        "Energy": SIMS_G_uniq, "Occp": occ_density_S, "Pair": SIMS_pair_uniq, "DP": SIMS_dict_uniq[:,0],"HT": SIMS_HT_uniq,
        "PCA 1": pca_coords[:,0], "PCA 2": pca_coords[:,1],
        "PCA X": pca_coords[:,0], "PCA Y": pca_coords[:,1], "PCA Z": pca_coords[:,2],
        "PHATE 1": phate_coords[:,0], "PHATE 2": phate_coords[:,1],
        "t-SNE 1": tsne_coord_2d[:,0],"t-SNE 2": tsne_coord_2d[:,1],
        "t-SNE X": tsne_coord_3d[:,0],"t-SNE Y": tsne_coord_3d[:,1],"t-SNE Z": tsne_coord_3d[:,2],
        "UMAP 1": umap_coord_2d[:,0],"UMAP 2": umap_coord_2d[:,1],
        "UMAP X": umap_coord_3d[:,0],"UMAP Y": umap_coord_3d[:,1],"UMAP Z": umap_coord_3d[:,2],
        }
        )

# make dataframe for plotting
dfall = pd.DataFrame(data={
        "Energy": SIMS_G, "Pair": SIMS_pair,"DP": SIMS_dict[:,0],"HT": SIMS_HT,"TotalT": SIMS_T,
        "PCA 1": pca_all_coords[:,0], "PCA 2": pca_all_coords[:,1],
        "PCA X": pca_all_coords[:,0], "PCA Y": pca_all_coords[:,1], "PCA Z": pca_all_coords[:,2],
        "PHATE 1": phate_all_coords[:,0], "PHATE 2": phate_all_coords[:,1],
        "t-SNE 1": tsne_all_coord_2d[:,0],"t-SNE 2": tsne_all_coord_2d[:,1],
        "t-SNE X": tsne_all_coord_3d[:,0],"t-SNE Y": tsne_all_coord_3d[:,1],"t-SNE Z": tsne_all_coord_3d[:,2],
        "UMAP 1": umap_all_coord_2d[:,0],"UMAP 2": umap_all_coord_2d[:,1],
        "UMAP X": umap_all_coord_3d[:,0],"UMAP Y": umap_all_coord_3d[:,1],"UMAP Z": umap_all_coord_3d[:,2],
        }
        )

df0 = df[df["Pair"] == 0]
df1 = df[df["Pair"] == 1]

dfall0 = dfall[dfall["Pair"] == 0]
dfall1 = dfall[dfall["Pair"] == 1]

## Visualize

### Interactive plots 2D

In [ ]:
# make interactive plotly 2D plot for different visualization methods
VIS_METHOD = ["PCA", "PHATE", "UMAP", "t-SNE"]
print(SEQ)

# for vis in VIS_METHOD: 
#     fig = interactive_plotly_2D(SEQ,df,dfall,trj_id,vis)
#     print("DONE: ", vis)
#     pio.write_html(fig, file="../interactive_plot/{}_{}_2D.html".format(SEQ,vis), auto_open=True)


vis = VIS_METHOD[0]
fig = interactive_plotly_2D(SEQ,df,dfall,trj_id,vis)
# fig.show()
pio.write_html(fig, file="{}_{}_2D.html".format(SEQ,vis), auto_open=True)

#### Seperate Paired and Unpaired

In [ ]:
# separate paired and unpaired data plots
def interactive_plotly_2D_01(SEQ,df,dfall,trj_id,vis):
    fig = go.Figure()
    
    # plot energy landscape background
    fig.add_trace(go.Scattergl(
            x=df0["{} 1".format(vis)], 
            y=df0["{} 2".format(vis)], 
            mode='markers',
            marker=dict(
                symbol='x',
                size=10,
                sizemode='area',
                # size=df["HT"],
                # sizeref=5e-11,
                color=df0["Energy"],
                # colorscale="Plasma",
                colorscale="OrRd",
                showscale=True,
                colorbar_x=1.2,
            ),
            customdata = np.stack((df0['Pair'],np.log(df0["Occp"]),df0["HT"]),axis=-1),
            text=df0['DP'],
            hovertemplate=
                "DP notation: <br> <b>%{text}</b><br><br>" +
                "X: %{x}   " + "   Y: %{y} <br>"+
                "Energy:  %{marker.color:.3f} kcal/mol<br>"+
                "Average Holding Time:  %{customdata[2]:.5g} s<br>"+
                "Occupancy Density (logscale):  %{customdata[1]:.3f}<br>"+
                "Pair (0/1->unpaired/paired,resp):  %{customdata[0]}",
            name="Unpaired",
            # showlegend=False,
        )
    )

    fig.add_trace(go.Scattergl(
            x=df1["{} 1".format(vis)], 
            y=df1["{} 2".format(vis)], 
            mode='markers',
            marker=dict(
                sizemode='area',
                size=4,
                # size=df["HT"],
                # sizeref=5e-11,
                color=df1["Energy"],
                # colorscale="Plasma",
                colorscale="Greens",
                opacity=0.5,
                showscale=True,
                colorbar_x=-0.2,
            ),
            customdata = np.stack((df1['Pair'],np.log(df1["Occp"]),df1["HT"]),axis=-1),
            text=df1['DP'],
            hovertemplate=
                "DP notation: <br> <b>%{text}</b><br><br>" +
                "X: %{x}   " + "   Y: %{y} <br>"+
                "Energy:  %{marker.color:.3f} kcal/mol<br>"+
                "Average Holding Time:  %{customdata[2]:.5g} s<br>"+
                "Occupancy Density (logscale):  %{customdata[1]:.3f}<br>"+
                "Pair (0/1->unpaired/paired,resp):  %{customdata[0]}",
            name="Paired",
            # showlegend=False,
        )
    )

    # layout trajectory on top of energy landscape
    for i in range(1):
        subdf = plot_trj(trj_id,dfall,i,vis,dim="2D")[0]
        fig.add_trace(
            go.Scattergl(
                x=subdf["sub X"], 
                y=subdf["sub Y"],
                mode='lines+markers',
                line=dict(
                    # color='rgb({}, {}, {})'.format((i/100*255),(i/100*255),(i/100*255)),
                    color="black",
                    width=2+i/100,
                ),
                marker=dict(
                    sizemode='area',
                    size=subdf["HT"],
                    sizeref=8e-11,
                    color=subdf["Energy"],
                    colorscale="plasma",
                    # showscale=False,
                    colorbar_orientation='h',
                ),
                
                text=subdf["Step"],
                customdata = np.stack((subdf['Pair'],subdf["TotalT"],subdf["DP"]),axis=-1),
                hovertemplate=
                    "Step:  <b>%{text}</b><br><br>"+
                    "X: %{x}   " + "   Y: %{y} <br>"+
                    "DP: %{customdata[2]}<br>" +
                    "Energy:  %{marker.color:.3f} kcal/mol<br>"+
                    "Average Holding Time:  %{marker.size:.5g} s<br>"+
                    "Total Time:  %{customdata[1]:.5e} s<br>" +
                    "Pair (0/1->unpaired/paired,resp):  %{customdata[0]} ",
                visible='legendonly'
                        )
        )

    # label initial and final states
    fig.add_trace(
        go.Scattergl(
            x=plot_trj(trj_id,dfall,i,vis,dim="2D")[1],
            y=plot_trj(trj_id,dfall,0,vis,dim="2D")[2],
            mode='markers+text',
            marker_color="lime", 
            marker_size=15,
            text=["I", "F"],
            textposition="middle center",
            textfont=dict(
            family="sans serif",
            size=16,
            color="black"
        ),
            hoverinfo='skip',
            showlegend=False,
                        )
    )

    fig.update_layout(
        title="{}: {} Vis".format(SEQ,vis),
        xaxis=dict(
                title="{} 1".format(vis),
            ),
        yaxis=dict(
                title="{} 2".format(vis),
            ),
        legend=dict(
            title="Single Trajectory",
            title_font=dict(size=10),
            font=dict(
                # family="Courier",
                size=10,
                color="black"
        )
        ),
        hovermode="x",
    )
    
    return fig


In [ ]:
vis = VIS_METHOD[0]
fig = interactive_plotly_2D_01(SEQ,df,dfall,trj_id,vis)
# fig.show()
pio.write_html(fig, file="{}_{}_2D.html".format(SEQ,vis), auto_open=True)

### Interactive plots 3D

In [ ]:
# make interactive plotly 3D plot for different visualization methods
VIS_METHOD = ["PCA", "UMAP", "t-SNE"]
print(SEQ)

# for vis in VIS_METHOD: 
#     fig = interactive_plotly_3D(SEQ,df,dfall,trj_id,vis)
#     print("DONE: ", vis)
#     pio.write_html(fig, file="../interactive_plot/{}_{}_3D.html".format(SEQ,vis), auto_open=True)
    

vis = VIS_METHOD[0]
fig = interactive_plotly_3D(SEQ,df,dfall,trj_id,vis)
# fig.show()
pio.write_html(fig, file="{}_{}_3D.html".format(SEQ,vis), auto_open=True)


### Self maded interactive plot (not use)

In [ ]:
%matplotlib qt

SEQ = "PT0"
fnpz_data = "data/helix_assoc/helix_assoc_{}_multrj_100epoch_py.npz".format(SEQ)
data_npz = np.load(fnpz_data)
# make plots
interactive_plot_old(data_npz,SEQ,vis="PHATE")

## Desity Coutour

#### Pair vs Unpaired Contour Density

In [ ]:
VIS_METHOD = ["PCA", "PHATE", "UMAP", "t-SNE"]

In [ ]:
# paired vs unpaired contuours
vis = VIS_METHOD[0]

fig = px.density_contour(dfall, x="{} 1".format(vis), y="{} 2".format(vis),
                        marginal_x="histogram", marginal_y="histogram",
                        color="Pair",
                        # facet_col="Pair",
                        title="{}".format(SEQ),
                        )
fig.update_layout(hovermode="x")

pio.write_html(fig, file="{}_desitycontour_pair.html".format(SEQ), auto_open=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Contour(
        x = dfall["PCA 1"],
        y = dfall["PCA 2"],
        z = dfall["PCA Z"],
        colorscale = 'Blues',
        # colorscale = dfall["Pair"],
        
))
pio.write_html(fig, file="{}_desitycontour.html".format(SEQ), auto_open=True)

In [ ]:
# # too large, not work well.
# # scatter layout lay on the Histogram2d plots
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=df["PCA 1"],
#     y=df["PCA 2"],
#     mode='markers',
#     showlegend=False,
#     marker=dict(
#         symbol='x',
#         opacity=0.7,
#         color='white',
#         size=8,
#         line=dict(width=1),
#     )
# ))
# # fig.add_trace(go.Scatter(
# #     x=x1,
# #     y=y1,
# #     mode='markers',
# #     showlegend=False,
# #     marker=dict(
# #         symbol='circle',
# #         opacity=0.7,
# #         color='white',
# #         size=8,
# #         line=dict(width=1),
# #     )
# # ))
# fig.add_trace(go.Histogram2d(
#     x=dfall["PCA 1"],
#     y=dfall["PCA 2"],
#     colorscale='YlGnBu',
#     zmax=10,
#     nbinsx=14,
#     nbinsy=14,
#     zauto=False,
# ))

# fig.update_layout(
#     xaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
#     yaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
#     autosize=False,
#     height=550,
#     width=550,
#     hovermode='closest',

# )

# pio.write_html(fig, file="{}_Histogram2d.html".format(SEQ), auto_open=True)

In [ ]:
# get unique structure density heatmap
vis = VIS_METHOD[0]
fig = px.density_heatmap(df, x=f"{vis} 1", y=f"{vis} 2",
                         text_auto=True,
                         title="Unique Structure Density Heatmap")
pio.write_html(fig, file="{}_density_heatmap_uniq.html".format(SEQ), auto_open=True)

## Draw helix structure

In [ ]:
import networkx as nx
from networkx.drawing.nx_pylab import draw_networkx
from networkx.drawing.layout import *
import matplotlib.pyplot as plt

In [ ]:
for j in range(-20,0,5):
    fig, ax = plt.subplots(1,5, figsize=(20,5))
    ex_neighborhood_graphs = [nx.convert_matrix.from_numpy_matrix(x) for x in SIMS_adj[j:j+5]]
    if j+5 == 0:
        ex_neighborhood_graphs = [nx.convert_matrix.from_numpy_matrix(x) for x in SIMS_adj[j:]]

    for i, g in enumerate(ex_neighborhood_graphs):
        nx.draw(g, ax=ax[i], node_size=30, with_labels=True, pos=kamada_kawai_layout(g),font_size=8)  # pos=circular_layout(g)


In [ ]:
s12 = "....((((.....(.....).....))..))..(...........)...."
print(s12,len(s12))

d_a2 = dot2adj(s12,helix=True)
print(s12,len(s12),d_a2[24,25])

g = nx.convert_matrix.from_numpy_matrix(d_a2)

nx.draw(g, node_size=90, pos=kamada_kawai_layout(g),with_labels=True,font_size=10) 